# Fine-Tuning kkirchheim/german-gpt2-medium on Battle Rap Data
This notebook demonstrates how to fine-tune the [kkirchheim/german-gpt2-medium](https://huggingface.co/kkirchheim/german-gpt2-medium) model using your `battles.csv` file containing German battle rap lyrics.

In [ ]:
# Install required libraries (uncomment if needed)
# !pip install transformers datasets accelerate

In [3]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 63.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [datasets]/19 [datasets]e]s]ub]


In [4]:
import pandas as pd
from datasets import Dataset

# Load your battle rap data
df = pd.read_csv('../battles.csv')

# For fine-tuning, we only need the text. Let's use the 'lyrics' column.
df = df.dropna(subset=['lyrics'])
dataset = Dataset.from_pandas(df[['lyrics']].rename(columns={'lyrics': 'text'}))

# Optionally, split into train/validation (here, all for training due to small size)
train_dataset = dataset

In [7]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

model_name = 'kkirchheim/german-gpt2-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/981 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./german-gpt2-battle-rap',
    per_device_train_batch_size=2,
    num_train_epochs=5,
    save_steps=10,
    save_total_limit=2,
    logging_steps=5,
    fp16=True,
    report_to='none',
    push_to_hub=False
)

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

## Inference Example
After training, you can generate new battle rap lyrics like this:

In [ ]:
prompt = "[Dein Battle-Rap-Start hier]"
inputs = tokenizer(prompt, return_tensors='pt')
outputs = model.generate(**inputs, max_length=200, do_sample=True, top_k=50, top_p=0.95)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))